# Submission Information:

### Team Member 1:
* UNI: ba2500
* Name: Baran Akyol

### Team Member 2 [optional]:
* UNI:  
* Name:

# Step0 - Import Libraries, Load Data [0 points]

This is the basic step where you can load the data and create train and test sets for internal validation as per your convinience.

In [485]:
#from universe import all
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import Binarizer
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import RandomizedSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.datasets import make_gaussian_quantiles
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomTreesEmbedding, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.kernel_approximation import RBFSampler
from imblearn.under_sampling import RandomUnderSampler
#from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from imblearn.under_sampling import CondensedNearestNeighbour
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MaxAbsScaler



In [441]:
#Training Data
data = pd.read_csv("data/data.csv")
del data['duration']


In [442]:
#Testing Data
holdout = pd.read_csv("data/holdout.csv")
#del holdout['duration']
ID = holdout['ID']
del holdout['ID']

# Step1 - Exploration and Preparation [10 points]

In this step, we expect you to look into the data and try to understand it before modeling. This understanding may lead to some basic data preparation steps which are common across the two model sets required.

In [443]:
# A sloppy function to give an overview of the distribution of variables

def count_columns(dataframe,rows=10):
    """
    parameters
    ----------
    dataframe: pandas DataFrame
    
    rows: Number of rows to display
    
    """
    
    if type(dataframe) != 'pandas.core.frame.DataFrame':
        dataframe = pd.DataFrame(dataframe)
    
    def singlecolumn(i):
        
        s = pd.DataFrame(dataframe[i].value_counts().index,columns=[i])
        
        s[len(dataframe[i].unique())] = dataframe[i].value_counts().values
        
        return s

    pd.options.display.max_columns= len(dataframe) * 2
    
    return pd.concat( [ singlecolumn(i) for i in dataframe.columns ],axis=1) \
                            .iloc[0:rows,:].replace(np.nan ,' ', regex=True)

count_columns(data,13)

,age,78,job,12,marital_status,4,education,8,credit_default,3,housing,3,loan,3,contact,2,month,10,day_of_week,5,campaign,40,prev_days,26,prev_contacts,8,prev_outcomes,3,emp_var_rate,32950,cons_price_idx,32950,cons_conf_idx,32950,euribor3m,32950,nr_employed,207,subscribed,2
0,33.0,1383.0,admin.,8342,married,19955,university.degree,9760,no,26059,yes,17176,no,27179,cellular,20902,may,11016,thu,6945,1.0,13235.0,999.0,31707.0,0,28423,nonexistent,28423,-1.702897,1,94.156527,1,-41.265896,1,4.968435,1,5229.0,1048.0,no,29238
1,34.0,1376.0,blue-collar,7408,single,9256,high.school,7580,unknown,6888,no,14974,yes,4971,telephone,12048,jul,5716,mon,6749,2.0,8682.0,3.0,373.0,1,3670,failure,3390,-1.871395,1,92.589502,1,-43.258444,1,5.070245,1,5228.0,1037.0,yes,3712
2,32.0,1295.0,technician,5371,divorced,3676,basic.9y,4889,yes,3,unknown,800,unknown,800,,,aug,4954,wed,6531,3.0,4531.0,6.0,332.0,2,606,success,1137,1.444970,1,92.786254,1,-42.295621,1,1.323636,1,5227.0,997.0,,
3,36.0,1275.0,services,3168,unknown,63,professional.course,4154,,,,,,,,,jun,4272,tue,6495,4.0,2386.0,4.0,105.0,3,170,,,-1.874953,1,93.094787,1,-36.077564,1,5.021518,1,5230.0,987.0,,
4,37.0,1265.0,management,2340,,,basic.4y,3313,,,,,,,,,nov,3299,fri,6230,5.0,1335.0,9.0,55.0,4,58,,,-3.260938,1,91.397460,1,-40.009586,1,4.245056,1,5226.0,923.0,,
5,31.0,1258.0,retired,1390,,,basic.6y,1846,,,,,,,,,apr,2097,,,6.0,798.0,7.0,48.0,5,17,,,1.075140,1,91.519714,1,-42.665607,1,4.818017,1,5225.0,881.0,,
6,35.0,1255.0,entrepreneur,1198,,,unknown,1394,,,,,,,,,oct,540,,,7.0,519.0,2.0,46.0,6,5,,,1.151109,1,92.128773,1,-36.791706,1,1.106071,1,5231.0,859.0,,
7,30.0,1187.0,self-employed,1140,,,illiterate,14,,,,,,,,,sep,465,,,8.0,330.0,12.0,45.0,7,1,,,1.310731,1,95.146087,1,-46.603565,1,1.295641,1,5232.0,777.0,,
8,38.0,1146.0,housemaid,827,,,,,,,,,,,,,mar,437,,,9.0,242.0,5.0,42.0,,,,,-1.922613,1,92.750438,1,-35.820636,1,0.886056,1,5224.0,728.0,,
9,39.0,1111.0,unemployed,797,,,,,,,,,,,,,dec,154,,,10.0,184.0,10.0,40.0,,,,,0.903708,1,91.262542,1,-46.581714,1,4.975101,1,5193.0,707.0,,


In [444]:
def select_columns(dataframe):
    Continuous = ['age','prev_days','emp_var_rate','cons_conf_idx','euribor3m','nr_employed']
    
    Categorical = ['job','marital_status','education','credit_default','housing',
                   'loan','contact','month','day_of_week','prev_outcomes','campaign']
    if 'subscribed' in dataframe.columns:
        return dataframe[Continuous + Categorical + ['subscribed']],Categorical
    else:
        return dataframe[Continuous + Categorical],Categorical

TRAIN,cat=select_columns(data)
HOLD,cat=select_columns(holdout)

Clearly some variables are categorical and may need one hot encoding. To not deal with pipeline encoding everytime I will add an option to my preproces function. I don't think I need to encode N/A values seperately as dummy encoding will encode them as well. Not all columns in the holdout set are in train and vice versa however so have to manually add those columns for compatibility.

In [445]:
def preprocess(dataframe,Categoricals,dummy=0):
   
    """Label Encode categorical variables using dummy variable and seperate frame to X,y
    
    Returns
    -------
    
    X : numpy.ndarray
        Label Encoded DataFrame consisting of features
         
    y : numpy.ndarray
        Response Variable, 1d array
    
    """
    
    data = dataframe.copy()

    data = data[data.credit_default != 'yes']
    
    
    if dummy == 1:
        #apply dummy variabl encoding
        data = pd.get_dummies(data,columns=Categoricals)
        
        # there are  values that don't match in categories section this is to manage that
        if 'campaign' in Categoricals and 'subscribed' in data.columns:
            data.insert(94,'campaign_36.0',0)
            data.insert(100,'campaign_43.0',0)

        elif 'campaign' in Categoricals:
            data.insert(85,'campaign_32.0',0)
            data.insert(85,'campaign_31.0',0)
            data.insert(85,'campaign_30.0',0)
            data.insert(85,'campaign_29.0',0)
            data.insert(90,'campaign_34.0',0)
            data.insert(93,'campaign_37.0',0)
            data.insert(94,'campaign_42.0',0)
            data.insert(94,'campaign_41.0',0)
            data.insert(94,'campaign_39.0',0)
            data.insert(97,'campaign_56.0',0)
            
            
            
    else:
        #apply labelencoding to every categorical column
        data[Categoricals]=data[Categoricals].apply(preprocessing.LabelEncoder().fit_transform)
    
    if 'subscribed' in data.columns:
        
        #extract y as binary column and
        y = preprocessing.LabelBinarizer().fit_transform(data['subscribed'])
        #delete y from original 
        del data['subscribed'] 
        return np.array(data),y.reshape((y.shape[0],))

    
    else:
        
        return np.array(data)

In [547]:
def cv_score(alg,X,y):
    
    """Computes cv score
    
    parameters
    ----------
    
    alg: The algorithm to use
    
    """
    
    score = cross_val_score(alg,X,y,cv=5,scoring='roc_auc',verbose=1)
    print 'scores : ', score
    tesprint 'mean : ', np.mean(score)
    return np.mean(score)

In [447]:
def construct_holdout(ID,prediction,):
    
    """Saves results as a kaggle ready .csv file
    
    """
    #result = pd.DataFrame(np.column_stack((ID,prediction)),columns=[float('ID'),'subscribed'])
    prediction = pd.DataFrame(prediction,columns=['subscribed'])
    prediction['ID'] = ID
    prediction.to_csv('holdout.csv',columns = ['ID','subscribed'],index=False,float_format='%f')
    
    print 'success'

In [448]:
TRAIN,cat=select_columns(data)
HOLD,cat=select_columns(holdout)
X, y = preprocess(TRAIN,cat)
H = preprocess(HOLD,cat)

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=2017)

In [449]:
DTRAIN,cat=select_columns(data)
DHOLD,cat=select_columns(holdout)
DX, Dy = preprocess(TRAIN,cat,dummy=1)
DH = preprocess(HOLD,cat,dummy=1)

DX_train,DX_test,Dy_train,Dy_test = train_test_split(X,y,random_state=2017)

# Step2 - ModelSet1 [35 points]

In this step, we expect you to perform the following steps relevant to the models you choose for set1:

* feature engineering
* validation
* feature selection
* final model selection

You may select up to 5 models in this step for the purpose of final ensemble. Any classification algorithm covered in class apart from tree-based models can be tested here.

In [451]:
# FIRST MODEL

# LogisticRegression

log=LogisticRegressionCV(fit_intercept=0)

combined_features = FeatureUnion([("FA", FactorAnalysis()), ("univ_select", SelectKBest(k=12))])

logregpipe = make_pipeline(MinMaxScaler((0,1)),SelectKBest(k=14),log)

#logregpipe.fit(X_train,y_train)

#cv_score(logregpipe,DX,Dy)  #0.79
"""
scores :  [ 0.78967346  0.78190539  0.79355691  0.80207903  0.78288818]
mean :  0.790020591501
"""



'\nscores :  [ 0.78967346  0.78190539  0.79355691  0.80207903  0.78288818]\nmean :  0.790020591501\n'

In [452]:
#SECOND MODEL

#RidgeClassifierCV


Ridge = make_pipeline(MinMaxScaler(),SelectKBest(k=15),RidgeClassifierCV(alphas=(0.1,0.25,0.4,0.7,1)))

#cv_score(Ridge,DX,Dy) 

"""
scores :  [ 0.7882516   0.77936115  0.79386554  0.79763645  0.77727998]
mean :  0.787278945004

"""

'\nscores :  [ 0.7882516   0.77936115  0.79386554  0.79763645  0.77727998]\nmean :  0.787278945004\n\n'

In [453]:
#THIRD MODEL

#GaussianNB()

#named it to pipe at the final second and its stuck as name so can't call it but didn't use it anyway

"""
GB = GaussianNB()

#cv_score(GNB,X,y) 

scores :  [ 0.77750643  0.76044588  0.77705859  0.78616237  0.771978  ]
mean :  0.774630254457

"""


'\nGB = GaussianNB()\n\n#cv_score(GNB,X,y) \n\nscores :  [ 0.77750643  0.76044588  0.77705859  0.78616237  0.771978  ]\nmean :  0.774630254457\n\n'

In [454]:
LDA = LinearDiscriminantAnalysis()

#cv_score(LDA,X,y)
"""
scores :  [ 0.7876034   0.7854583   0.79613523  0.79419745  0.77840803]
mean :  0.788360483811
"""

'\nscores :  [ 0.7876034   0.7854583   0.79613523  0.79419745  0.77840803]\nmean :  0.788360483811\n'

# Step3 - ModelSet2 [35 points]

In this step, we expect you to perform the following steps relevant to the models you choose for set2:

* feature engineering
* validation
* feature selection
* final model selection

You may select up to 5 models in this step for the purpose of final ensemble. We encourage you to try decition tree, random forest and gradient boosted tree methods here and pick the one which you think works best.

In [457]:
#FIRST MODEL

#XGBoost

xgbClassifier = xgb.XGBClassifier(
 learning_rate =0.05,
 n_estimators=57,
 max_depth=6,
 min_child_weight=3,
 gamma=0.2,
 subsample=0.75,
 colsample_bytree=0.75,
 objective= 'binary:logistic',
 nthread=3,
 scale_pos_weight=1,
 seed=42
)

dtrain = xgb.DMatrix(DX,Dy)


# Below are cv and gridsearch I used to tune the parameters
"""
param = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch1.fit(X,y)
gsearch1.best_params_
"""

"""
#check to see which step produces best results
xgb.cv(xgbClassifier.get_xgb_params(), dtrain, num_boost_round=xgbClassifier.get_params()['n_estimators'], nfold=5,
            metrics= {'auc'}, show_progress=True)
"""


"\n#check to see which step produces best results\nxgb.cv(xgbClassifier.get_xgb_params(), dtrain, num_boost_round=xgbClassifier.get_params()['n_estimators'], nfold=5,\n            metrics= {'auc'}, show_progress=True)\n"

In [458]:
#cv_score(xgbClassifier,DX,Dy) #0.797

In [534]:
# SECOND MODEL RANDOM FOREST CLASSIFIER

#gridsearch for parameters
"""
param_grid = {"max_depth": [3, None],
              "min_samples_split": [2, 5, 10],
              "min_samples_leaf": [2, 5, 10],
              "bootstrap": [True, False]}

rfgrid = GridSearchCV(RandomForestClassifier(n_estimators=180), param_grid=param_grid)
rfgrid.fit(X_train,y_train)
"""

rf = RandomForestClassifier(n_estimators=180,criterion='entropy',max_depth = 9,\
                              max_features=15,min_samples_split=2,min_samples_leaf=3,warm_start=1)

#rf.fit(X,y)


In [460]:
#cv_score(rf,DX,Dy)  #0.797

In [461]:
#THIRD MODEL EXTRA TREES CLASSIFIER

extratrees = ExtraTreesClassifier(n_estimators = 180, criterion='entropy',max_features=12,max_depth=8,min_samples_split=3,min_samples_leaf=4,warm_start=1)

#extratrees.fit(X,y)


In [462]:
#cv_score(extratrees,X,y) #0.799

In [463]:
#FOURTH MODEL GRADIENT BOOSTING CLASSIFIER

GBM = GradientBoostingClassifier(n_estimators=110, learning_rate=0.3,max_features = 20,
        max_depth=2, random_state=0)
#GBM.fit(X_train, y_train)

In [464]:
#cv_score(GBM,X,y) #0.797

# Step4 - Ensemble [20 points + 10 Bonus points]

In this step, we expect you to use the models created before and create new predictions. You should definitely try poor man's stacking but we encourage you to think of different ensemble techniques as well. We will judge your creativity and improvement in model performance using ensemble models and you can potentially earn 10 bonus points here.

In [465]:

#Voting Classifier using homogenous voting

Vhard = VotingClassifier(estimators=[('lr', logregpipe), ('rf', rf), ('xgb', xgbClassifier)], voting='soft')

#doesn't work well

In [543]:

#Voting Classifier using weighted votes

Vweighted = VotingClassifier(estimators=[('lr', logregpipe), ('rf', rf), \
                                    ('xgb', xgbClassifier)], voting='soft', weights=[3,19,5])

#works better  cv around 0.7982  which is highest score I reached till now

In [467]:
# Another Ensemble

Vweighted2 = VotingClassifier(estimators=[('lr', logregpipe),('rf', rf), \
                                  ('extra',extratrees),  ('xgb', xgbClassifier)], voting='soft', weights=[2,20,7,2])

#tried tuning the weights but no results

In [567]:
# STACKING

#doesn't work at all

"""also I couldn't install this on travis so quaoting it out """
#lr = LogisticRegression()
#sclf = StackingClassifier(classifiers=[logregpipe, rf, xgbClassifier], 
                         # meta_classifier=lr)

"""
cv_score(sclf,DX,Dy)

scores :  [ 0.61367795  0.61611216  0.61312319  0.6060669   0.62375434]
mean :  0.614546908301


"""

'\ncv_score(sclf,DX,Dy)\n\nscores :  [ 0.61367795  0.61611216  0.61312319  0.6060669   0.62375434]\nmean :  0.614546908301\n\n\n'

In [473]:
# Under Sample

#doesn't improve results

UnderSample = make_imb_pipeline(RandomUnderSampler(random_state=0, replacement=True),rf)

#cv_score(UnderSample,DX,Dy)  0.79715375352   # no improvement

In [475]:
US2 = make_imb_pipeline(RandomUnderSampler(random_state=0, replacement=True),logregpipe)
US3 = make_imb_pipeline(RandomUnderSampler(random_state=0, replacement=True),xgbClassifier)
US4 = make_imb_pipeline(RandomUnderSampler(random_state=0, replacement=True),extratrees)


USVote = VotingClassifier(estimators=[('1',UnderSample),('2',US2),('3',US3),('4',US4)],voting='soft')

"""
scores :  [ 0.79953921  0.79011081  0.796847    0.80815789  0.78773781]
mean :  0.796478545972
"""

#No improvement

'\nscores :  [ 0.79953921  0.79011081  0.796847    0.80815789  0.78773781]\nmean :  0.796478545972\n'

In [476]:
#Condensed Nearest Neighbors

cnn = make_imb_pipeline(CondensedNearestNeighbour(),rf)

# cv_score(cnn,DX,Dy)

# Takes way too long with no significant increase

In [477]:
#Took my best estimator, changed 1 parameter for each keeping others consant

rf1 = RandomForestClassifier(n_estimators=180,criterion='entropy',max_depth = 9,\
                              max_features=15,min_samples_split=2,min_samples_leaf=3,warm_start=1)
rf2 = RandomForestClassifier(n_estimators=90,criterion='entropy',max_depth = 9,\
                              max_features=15,min_samples_split=2,min_samples_leaf=3,warm_start=1)
rf3 = RandomForestClassifier(n_estimators=180,criterion='entropy',max_depth = 4,\
                              max_features=15,min_samples_split=2,min_samples_leaf=3,warm_start=1)
rf4 = RandomForestClassifier(n_estimators=180,criterion='gini',max_depth = 9,\
                              max_features=15,min_samples_split=2,min_samples_leaf=3,warm_start=0)
rf5 = RandomForestClassifier(n_estimators=180,criterion='entropy',max_depth = 9,max_features=15,min_samples_split=5,min_samples_leaf=3,warm_start=1)
rf6 = RandomForestClassifier(n_estimators=180,criterion='entropy',max_depth = 9,\
                              max_features=9,min_samples_split=2,min_samples_leaf=3,warm_start=1)
rf7 = RandomForestClassifier(n_estimators=180,criterion='entropy',max_depth = 9,max_features=15,min_samples_split=2,min_samples_leaf=7,warm_start=1)
rf8 = RandomForestClassifier(n_estimators=180,criterion='entropy',max_depth = 9,\
                              max_features=15,min_samples_split=2,min_samples_leaf=1,warm_start=1)


rfVote = VotingClassifier(estimators = [('rf1',rf1),('rf2',rf2),('rf3',rf3),('rf4',rf4),('rf5',rf5),\
                                       ('rf6',rf6),('rf7',rf7),('rf8',rf8)],voting='soft')

#cv_score(USVote,DX,Dy)

"""
cv_score(rfVote,DX,Dy)
scores :  [ 0.80212443  0.79096503  0.79649204  0.8073666   0.79172746]
mean :  0.7977351
"""

#No Improvement

'\ncv_score(rfVote,DX,Dy)\nscores :  [ 0.80212443  0.79096503  0.79649204  0.8073666   0.79172746]\nmean :  0.7977351\n'

In [528]:

# embed Random Forest and then use a classifier

rf_enc = OneHotEncoder()

rf_lm = xgbClassifier

rf.fit(X_train, y_train)

rf_enc.fit(rf.apply(X_train))

rf_lm.fit(rf_enc.transform(rf.apply(X_test)), y_test)

y_pred_rf_lm = rf_lm.predict_proba(rf_enc.transform(rf.apply(H)))[:, 1]

# FINAL PREDICTION USING THIS METHOD DID NOT PERFORM WELL ON KAGGLE


# Final Remarks

-For both ModelSet1 and ModelSet2, I was able to reach Kaggle benchmark. I tried imputation, scaling,
decomposition such as Factor Analysis or PCA and many other things but got no significant increase in my score so dropped most of the things I tried in the end. 

-I split my Training set further to X_train, X_test, etc. and evaluated models based on the X_train Cross Validation score initially. Later I started evaluating on X,y in order to fit the training set best and achieve a good Kaggle score.

-I commented most functional elements as TravisCI doesn't enjoy being overworked and at this point I respect it way too much to overwork it. 

-My Final Ensemble performs slightly better than my best performing classifier with a test score around 7.84, which is greater than 0.78 the benchmark for my other models. 

Please note that this is not the crossvalidation score where some models can perform higher.

In [563]:
Vweighted.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', Pipeline(steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selectkbest', SelectKBest(k=14, score_func=<function f_classif at 0x1142e4230>)), ('logisticregressioncv', LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercep...stic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=42, silent=True, subsample=0.75))],
         n_jobs=1, voting='soft', weights=[3, 19, 5])

In [553]:
pred= Vweighted.predict_proba(X_test)

In [ ]:
score = roc_auc_score(y_test,pred[:,1])

In [565]:
assert score > 0.78

0.78404063859813333